In [12]:
import pandas as pd
import adata

etfs = ['561300', '159726', '515100', '513500', '161119', '518880', '164824', '159985', '513330', '513100', '513030', '513520']

df = pd.DataFrame()

end_date = '2025-07-31'

for i in etfs:
    etf_df = adata.fund.market.get_market_etf(i, start_date='2022-01-01', end_date=end_date, k_type=1)
    etf_df['close'] = etf_df['close'].astype(float)
    etf_df['symbol'] = i
    etf_df = etf_df.ffill()
    etf_df = etf_df.infer_objects(copy=False)
    df = pd.concat([df, etf_df])

/tmp/ipykernel_97968/3288353483.py:14: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/tmp/ipykernel_97968/3288353483.py:14: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/tmp/ipykernel_97968/3288353483.py:14: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/tmp/ipykernel_97968/3288353483.py:14: FutureWarning:

Downcasting object dtype array

In [13]:
df_pivot = df.pivot(index='trade_date', columns='fund_code', values='close')
df_pivot = df_pivot.dropna()
df_pivot

fund_code,159726,159985,161119,164824,513030,513100,513330,513500,513520,515100,518880,561300
trade_date,,,,,,,,,,,,
2022-01-04,0.987,1.245,1.522,1.240,1.200,1.081,0.563,1.449,1.191,1.108,3.620,0.957
2022-01-05,0.994,1.262,1.521,1.234,1.194,1.059,0.541,1.443,1.190,1.098,3.635,0.946
2022-01-06,0.990,1.232,1.521,1.217,1.184,1.030,0.535,1.409,1.158,1.117,3.614,0.935
2022-01-07,1.014,1.234,1.519,1.235,1.186,1.038,0.553,1.418,1.158,1.123,3.592,0.934
2022-01-10,1.032,1.237,1.520,1.241,1.183,1.028,0.566,1.408,1.158,1.132,3.599,0.937
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-25,1.294,1.958,1.759,1.520,1.900,1.666,0.526,2.164,1.555,1.557,7.409,0.885
2025-07-28,1.289,1.943,1.759,1.515,1.917,1.684,0.526,2.185,1.534,1.549,7.388,0.887
2025-07-29,1.289,1.939,1.758,1.506,1.883,1.683,0.524,2.184,1.530,1.545,7.363,0.890


In [14]:
from sklearn import set_config
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split
)
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform
import matplotlib.pyplot as plt

from skfolio import RatioMeasure, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.distance import KendallDistance
from skfolio.model_selection import (
    CombinatorialPurgedCV,
    WalkForward,
    cross_val_predict,
)
from skfolio.moments import (
    DenoiseCovariance,
    DetoneCovariance,
    EWMu,
    GerberCovariance,
    ShrunkMu,
    ShrunkCovariance
)
from skfolio.optimization import (
    MeanRisk,
    NestedClustersOptimization,
    ObjectiveFunction,
    RiskBudgeting,
    HierarchicalRiskParity,
    DistributionallyRobustCVaR,
    StackingOptimization,
    MaximumDiversification,
    HierarchicalEqualRiskContribution,
    InverseVolatility,
    ConvexOptimization
)
from skfolio.pre_selection import SelectKExtremes
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import BlackLitterman, EmpiricalPrior, FactorModel
from skfolio.uncertainty_set import BootstrapMuUncertaintySet
from skfolio.portfolio import MultiPeriodPortfolio
from skfolio.cluster import HierarchicalClustering, LinkageMethod

In [15]:
prices = df_pivot.copy()
prices = prices.sort_values(by='trade_date').dropna()

cv = WalkForward(test_size=1, train_size=7, freq="MS")


model = HierarchicalRiskParity(
    risk_measure=RiskMeasure.CVAR,
    prior_estimator=EmpiricalPrior(
        covariance_estimator=ShrunkCovariance(shrinkage=0.9)
    )
)

model3 = RiskBudgeting(
    risk_measure=RiskMeasure.EVAR,
    portfolio_params=dict(name="Risk Parity - Covariance Shrinkage"),
    solver='SCS'
)

estimators = [
    ("model1", model),
    ("model2", MaximumDiversification()),
    ("model3", DistributionallyRobustCVaR(wasserstein_ball_radius=0.01))
]

model_stacking = StackingOptimization(
    estimators=estimators,
    final_estimator=model3,
)

X = prices_to_returns(prices)
X.index = pd.to_datetime(X.index)

pred_stacking = cross_val_predict(
    model_stacking,
    X,
    cv=cv,
    n_jobs=-1,
    portfolio_params=dict(name="Stacking")
)

In [16]:
returns = pred_stacking.plot_cumulative_returns()
returns.show(renderer='iframe')

In [17]:
composition = pred_stacking.plot_composition()
composition.show(renderer='iframe')

In [18]:
pred_stacking.summary()

Mean                                     0.037%
Annualized Mean                           9.33%
Variance                               0.00074%
Annualized Variance                       0.19%
Semi-Variance                          0.00041%
Annualized Semi-Variance                  0.10%
Standard Deviation                        0.27%
Annualized Standard Deviation             4.31%
Semi-Deviation                            0.20%
Annualized Semi-Deviation                 3.22%
Mean Absolute Deviation                   0.19%
CVaR at 95%                               0.57%
EVaR at 95%                               1.50%
Worst Realization                         2.62%
CDaR at 95%                               2.12%
MAX Drawdown                              3.13%
Average Drawdown                          0.51%
EDaR at 95%                               2.36%
First Lower Partial Moment               0.096%
Ulcer Index                              0.0076
Gini Mean Difference                    

In [19]:
pred_stacking.weights_per_observation

,159726,159985,161119,164824,513030,513100,513330,513500,513520,515100,518880,561300
2022-09-01,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-02,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-05,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-06,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
2022-09-07,0.018662,0.028534,0.670745,0.053054,0.021103,0.021200,0.013135,0.017874,0.019541,0.017503,0.101688,0.016961
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-24,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614
2025-06-25,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614
2025-06-26,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614
2025-06-27,0.016205,0.058234,0.676137,0.030993,0.030080,0.009944,0.027933,0.016175,0.022993,0.042024,0.045668,0.023614


In [20]:
df_c = df_pivot.copy()
df_c = prices_to_returns(df_c)
df_c.index = pd.to_datetime(df_c.index)

end_date = df_c.index.max()

train_start_date = (end_date - pd.DateOffset(months=7))
train_end_date = (end_date - pd.DateOffset(months=1))

train_df = df_c.loc[train_start_date:train_end_date]
predict_df = df_c.loc[train_end_date:]

model_stacking.fit(train_df)

pred_weight = model_stacking.predict(predict_df)

In [21]:
train_start_date

Timestamp('2024-12-31 00:00:00')

In [22]:
weight_df = pred_weight.weights_per_observation.tail(6)
weight_df.round(2)

,159726,159985,161119,164824,513030,513100,513330,513500,513520,515100,518880,561300
2025-07-24,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-25,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-28,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-29,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-30,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
2025-07-31,0.02,0.06,0.67,0.03,0.03,0.01,0.03,0.02,0.02,0.04,0.04,0.03
